In [50]:
import pandas as pd
import random
from datetime import datetime, timedelta
from scipy.stats import ttest_ind
from scipy.stats import mannwhitneyu
from scipy.stats import fisher_exact
from scipy.stats import chi2_contingency

## На платформе Kaggle или на Github найдите достаточно большой датасет с логами пользования сервисов (просмотры Youtube или стриминговых сервисов, такси/каршеринг или что угодно, где есть много юзеров и история их взаимодействия с каким-нибудь сервисом). В случае, если такого датасета нет - сгенерируйте самостоятельно (например, как на лекции было показано распределение по просмотру в Кион).

# Подготовка данных

Набор данных включает в себя такую ​​информацию, как 

* тип подписки пользователя (Basic, Standard или Premium)
* ежемесячный доход, полученный от его подписки
* дата присоединения к Netflix (Join Date)
* дата последнего платежа (Last Payment Date)
* страна, в которой он находится
* Возраст
* Пол
* тип устройства (например, Smart TV, мобильный телефон, настольный компьютер, планшет)
* Продолжительность подписки (я так поняла у всех стандартная 1 месяц)

In [2]:
df = pd.read_csv("/Users/tatianasalamatova/reps/MTS_data_analysis/stats_04/Netflix_Userbase.csv", sep = ",")
df

,User ID,Subscription Type,Monthly Revenue,Join Date,Last Payment Date,Country,Age,Gender,Device,Plan Duration
0,1,Basic,10,15-01-22,10-06-23,United States,28,Male,Smartphone,1 Month
1,2,Premium,15,05-09-21,22-06-23,Canada,35,Female,Tablet,1 Month
2,3,Standard,12,28-02-23,27-06-23,United Kingdom,42,Male,Smart TV,1 Month
3,4,Standard,12,10-07-22,26-06-23,Australia,51,Female,Laptop,1 Month
4,5,Basic,10,01-05-23,28-06-23,Germany,33,Male,Smartphone,1 Month
...,...,...,...,...,...,...,...,...,...,...
2495,2496,Premium,14,25-07-22,12-07-23,Spain,28,Female,Smart TV,1 Month
2496,2497,Basic,15,04-08-22,14-07-23,Spain,33,Female,Smart TV,1 Month
2497,2498,Standard,12,09-08-22,15-07-23,United States,38,Male,Laptop,1 Month
2498,2499,Standard,13,12-08-22,12-07-23,Canada,48,Female,Tablet,1 Month


*Генерирую случайных пользователей, чтобы сделать датасет размером 100к уникальных пользователей*

In [3]:
# Сначала получу уникальные возможные значения в каждом столбце:
Subscription_Type = df['Subscription Type'].unique()
Monthly_Revenue = df['Monthly Revenue'].unique()
Country = df['Country'].unique()
Age_max = max(df['Age'])
Age_min = min(df['Age'])
Device = df['Device'].unique()
max_user_id = max(df['User ID'])

# Join Date - буду генерировать случайно в диапазоне с 21 по 23 год
# Last Payment Date	- буду генерировать случайно в течение 23 года с июня по июль
# Age - буду генерировать случайно от макс до мин
# Gender - очевидно, что случайно
# Plan duration - у всех один = 1 month

Генерирую случайную дату в заданном диапазоне, которая возвращает случайную дату в заданном интервале между двумя датами.

In [4]:
def random_date(start, end):
    return start + timedelta(seconds=random.randint(0, int((end - start).total_seconds())))

Генерирую пользователя:

In [5]:
def generate_user(user_id):
    join_date = random_date(datetime(2021, 1, 1), datetime(2023, 12, 31))  # буду генерировать случайно в диапазоне с 21 по 23 год
    last_payment_date = random_date(datetime(2023, 6, 1), datetime(2023, 7, 31))  # буду генерировать случайно в течение 23 года  с июня по июль
    age = random.randint(Age_min, Age_max)  # буду генерировать случайно от макс до мин
    gender = random.choice(['Male', 'Female'])  # случайный пол
    
    return {
        'User ID': user_id,
        'Subscription Type': random.choice(Subscription_Type),
        'Monthly Revenue': random.choice(Monthly_Revenue),
        'Join Date': join_date.strftime('%d-%m-%y'),
        'Last Payment Date': last_payment_date.strftime('%d-%m-%y'),
        'Country': random.choice(Country),
        'Age': age,
        'Gender': gender,
        'Device': random.choice(Device),
        'Plan Duration': '1 Month'  # У всех один и тот же план
    }

In [6]:
new_users = [generate_user(i) for i in range(max_user_id + 1, max_user_id + 195001)]
new_users_df = pd.DataFrame(new_users)

Присоединяю таблицу к имеющейся:

In [7]:
df_analysis = pd.concat([df, new_users_df], ignore_index=True)
df_analysis.head(10)

,User ID,Subscription Type,Monthly Revenue,Join Date,Last Payment Date,Country,Age,Gender,Device,Plan Duration
0,1,Basic,10,15-01-22,10-06-23,United States,28,Male,Smartphone,1 Month
1,2,Premium,15,05-09-21,22-06-23,Canada,35,Female,Tablet,1 Month
2,3,Standard,12,28-02-23,27-06-23,United Kingdom,42,Male,Smart TV,1 Month
3,4,Standard,12,10-07-22,26-06-23,Australia,51,Female,Laptop,1 Month
4,5,Basic,10,01-05-23,28-06-23,Germany,33,Male,Smartphone,1 Month
5,6,Premium,15,18-03-22,27-06-23,France,29,Female,Smart TV,1 Month
6,7,Standard,12,09-12-21,25-06-23,Brazil,46,Male,Tablet,1 Month
7,8,Basic,10,02-04-23,24-06-23,Mexico,39,Female,Laptop,1 Month
8,9,Standard,12,20-10-22,23-06-23,Spain,37,Male,Smartphone,1 Month
9,10,Premium,15,07-01-23,22-06-23,Italy,44,Female,Smart TV,1 Month


## Разбейте на сегменты вашу аудиторию и примените для нескольких (примерно 3-4 метрики) метрик тесты Стьюдента, Манна-Уитни, Фишера (Фишера для нескольких сегментов). Проинтерпретируйте результаты, сделайте первые выводы.

Сегментирую по типу устройства: ['Smartphone', 'Tablet', 'Smart TV', 'Laptop'], получу 4 группы

Выберу метрики: 

* Monthly Revenue (с какого типа устройства пользователи приносят бОльший доход?)
* Subscription Type (на каком устройстве какой тип подписки?)
* Age (одинаков ли средний возраст для разных устройств?)

**Тест Стьюдента: проверю, есть ли статзначимые различия в средних значениях для каждого сегмента по попарному сравнению каждой из метрик?**

Мне нужно проводить тесты Стьюдента, но чтобы не растить ошибку второго рода (из - за проведения попраных сравнений), выберу 4 сегмента (это аргументация почему я взяла по типу устройства). Тогда вероятность ошибки второго рода = $1 - 0.95^4 = 18.5 \% $. При установленом уровене значимости (α), который обычно равен 0.05

Цель: Проверить, отличаются ли средние значения между двумя группами. У меня всего, получается 6 попарных сравнений. Буду сравнивать так: 

1) Smartphone vs Tablet
2) Smartphone vs Smart TV
3) Smartphone vs Laptop
4) Tablet vs Smart TV
5) Tablet vs Laptop
6) Smart TV vs Laptop

### Сравнение №1 - для метрики Monthly Revenue (с какого типа устройства пользователи приносят бОльший доход?)

Нулевая гипотеза (H0): Средние значения для метрики, указывающей на доход, который приносят пользователи в двух группах одинаковые.

Альтернативная гипотеза (H1): Средние значения в двух группах различаются.

Если p-value < 0.05 => разница статистически значимая, и нулевая гипотеза отвергается

Если p-value больше или равно 0.05 => не отвергаем нулевую гипотезу

In [8]:
# Сегрегация по типу устройства для MR
smartphone_revenue = df_analysis[df_analysis['Device'] == 'Smartphone']['Monthly Revenue']
tablet_revenue = df_analysis[df_analysis['Device'] == 'Tablet']['Monthly Revenue']
smart_tv_revenue = df_analysis[df_analysis['Device'] == 'Smart TV']['Monthly Revenue']
laptop_revenue = df_analysis[df_analysis['Device'] == 'Laptop']['Monthly Revenue']

# Попарные сравнения:
# Smartphone vs Tablet
t_stat, p_value = ttest_ind(smartphone_revenue, tablet_revenue)
if p_value < 0.05:
    print(f"Гипотезу о равенстве средних по доходу для Smartphone vs Tablet ОТВЕРГАЕМ: T-stat = {t_stat}, P-value = {p_value}")
else:
    print(f"Гипотезу о равенстве средних по доходу для Smartphone vs Tablet НЕ ОТВЕРГАЕМ: T-stat = {t_stat}, P-value = {p_value}")

# Smartphone vs Smart TV
t_stat, p_value = ttest_ind(smartphone_revenue, smart_tv_revenue)
if p_value < 0.05:
    print(f"Гипотезу о равенстве средних по доходу для Smartphone vs Smart TV ОТВЕРГАЕМ: T-stat = {t_stat}, P-value = {p_value}")
else:
    print(f"Гипотезу о равенстве средних по доходу для Smartphone vs Smart TV НЕ ОТВЕРГАЕМ: T-stat = {t_stat}, P-value = {p_value}")

# Smartphone vs Laptop
t_stat, p_value = ttest_ind(smartphone_revenue, laptop_revenue)
if p_value < 0.05:
    print(f"Гипотезу о равенстве средних по доходу для Smartphone vs Laptop ОТВЕРГАЕМ: T-stat = {t_stat}, P-value = {p_value}")
else:
    print(f"Гипотезу о равенстве средних по доходу для Smartphone vs Laptop НЕ ОТВЕРГАЕМ: T-stat = {t_stat}, P-value = {p_value}")


# Tablet vs Smart TV
t_stat, p_value = ttest_ind(tablet_revenue, smart_tv_revenue)
if p_value < 0.05:
    print(f"Гипотезу о равенстве средних по доходу для Tablet vs Smart TV ОТВЕРГАЕМ: T-stat = {t_stat}, P-value = {p_value}")
else:
    print(f"Гипотезу о равенстве средних по доходу для Tablet vs Smart TV НЕ ОТВЕРГАЕМ: T-stat = {t_stat}, P-value = {p_value}")

# Tablet vs Laptop
t_stat, p_value = ttest_ind(tablet_revenue, laptop_revenue)
if p_value < 0.05:
    print(f"Гипотезу о равенстве средних по доходу для Tablet vs Laptop ОТВЕРГАЕМ: T-stat = {t_stat}, P-value = {p_value}")
else:
    print(f"Гипотезу о равенстве средних по доходу для Tablet vs Laptop НЕ ОТВЕРГАЕМ: T-stat = {t_stat}, P-value = {p_value}")

# Smart TV vs Laptop
t_stat, p_value = ttest_ind(smart_tv_revenue, laptop_revenue)
if p_value < 0.05:
    print(f"Гипотезу о равенстве средних по доходу для Smart TV vs Laptop ОТВЕРГАЕМ: T-stat = {t_stat}, P-value = {p_value}")
else:
    print(f"Гипотезу о равенстве средних по доходу для Smart TV vs Laptop НЕ ОТВЕРГАЕМ: T-stat = {t_stat}, P-value = {p_value}")


Гипотезу о равенстве средних по доходу для Smartphone vs Tablet НЕ ОТВЕРГАЕМ: T-stat = -0.17518501085647348, P-value = 0.8609346205268489
Гипотезу о равенстве средних по доходу для Smartphone vs Smart TV НЕ ОТВЕРГАЕМ: T-stat = 0.5369438204466301, P-value = 0.5913076318793521
Гипотезу о равенстве средних по доходу для Smartphone vs Laptop НЕ ОТВЕРГАЕМ: T-stat = -1.0375419115920497, P-value = 0.29948591282762643
Гипотезу о равенстве средних по доходу для Tablet vs Smart TV НЕ ОТВЕРГАЕМ: T-stat = 0.7122637561951747, P-value = 0.4763031468437954
Гипотезу о равенстве средних по доходу для Tablet vs Laptop НЕ ОТВЕРГАЕМ: T-stat = -0.8633517762987598, P-value = 0.38794617932163167
Гипотезу о равенстве средних по доходу для Smart TV vs Laptop НЕ ОТВЕРГАЕМ: T-stat = -1.5722349737167507, P-value = 0.11589927520396147


Итак, на основании полученных данных, можно сделать вывод, что нулевая гипотеза ни в одном из случаев не была отвергнута. $=>$ статистически значимых различий в среднем доходе между пользователями смартфонов, планшетов, Smart TV и ноутбуков нет. На основании данных можно заключить, что тип устройства, используемого пользователями, не оказывает существенного влияния на средний доход, который они приносят компании. Это может быть связано с тем, что пользователи всех устройств оформляют схожие подписки, вне зависимости от устройства, которое они используют.

### Сравнение №2 - для метрики Subscription Type (на каком устройстве какой тип подписки?)

Для этого переведу подписки в баллы, чтобы было удобенее сравнивать. Пусть Basic = 1, Standard = 2, Premium = 3

Нулевая гипотеза (H0): Средний балл подписки одинаков для двух типов устройств.

Альтернативная гипотеза (H1): Средний "балл подписки" различается для двух типов устройств.

In [12]:
# дам каждому типу подписки балл (сделаю доп столбец)
df_analysis['Subscription Score'] = df_analysis['Subscription Type'].map({'Basic': 1, 'Standard': 2, 'Premium': 3})

smartphone_sub = df_analysis[df_analysis['Device'] == 'Smartphone']['Subscription Score']
tablet_sub = df_analysis[df_analysis['Device'] == 'Tablet']['Subscription Score']
smart_tv_sub = df_analysis[df_analysis['Device'] == 'Smart TV']['Subscription Score']
laptop_sub = df_analysis[df_analysis['Device'] == 'Laptop']['Subscription Score']

# Попарные сравнения:
# Smartphone vs Tablet
t_stat, p_value = ttest_ind(smartphone_sub, tablet_sub)
if p_value < 0.05:
    print(f"Гипотезу о равенстве средних баллов подписки для Smartphone vs Tablet ОТВЕРГАЕМ: T-stat = {t_stat}, P-value = {p_value}")
else:
    print(f"Гипотезу о равенстве средних баллов подписки для Smartphone vs Tablet НЕ ОТВЕРГАЕМ: T-stat = {t_stat}, P-value = {p_value}")

# Smartphone vs Smart TV
t_stat, p_value = ttest_ind(smartphone_sub, smart_tv_sub)
if p_value < 0.05:
    print(f"Гипотезу о равенстве средних баллов подписки для Smartphone vs Smart TV ОТВЕРГАЕМ: T-stat = {t_stat}, P-value = {p_value}")
else:
    print(f"Гипотезу о равенстве средних баллов подписки для Smartphone vs Smart TV НЕ ОТВЕРГАЕМ: T-stat = {t_stat}, P-value = {p_value}")

# Smartphone vs Laptop
t_stat, p_value = ttest_ind(smartphone_sub, laptop_sub)
if p_value < 0.05:
    print(f"Гипотезу о равенстве средних баллов подписки для Smartphone vs Laptop ОТВЕРГАЕМ: T-stat = {t_stat}, P-value = {p_value}")
else:
    print(f"Гипотезу о равенстве средних баллов подписки для Smartphone vs Laptop НЕ ОТВЕРГАЕМ: T-stat = {t_stat}, P-value = {p_value}")


# Tablet vs Smart TV
t_stat, p_value = ttest_ind(tablet_sub, smart_tv_sub)
if p_value < 0.05:
    print(f"Гипотезу о равенстве средних баллов подписки для Tablet vs Smart TV ОТВЕРГАЕМ: T-stat = {t_stat}, P-value = {p_value}")
else:
    print(f"Гипотезу о равенстве средних баллов подписки для Tablet vs Smart TV НЕ ОТВЕРГАЕМ: T-stat = {t_stat}, P-value = {p_value}")

# Tablet vs Laptop
t_stat, p_value = ttest_ind(tablet_sub, laptop_sub)
if p_value < 0.05:
    print(f"Гипотезу о равенстве средних баллов подпискидля Tablet vs Laptop ОТВЕРГАЕМ: T-stat = {t_stat}, P-value = {p_value}")
else:
    print(f"Гипотезу о равенстве средних баллов подписки для Tablet vs Laptop НЕ ОТВЕРГАЕМ: T-stat = {t_stat}, P-value = {p_value}")

# Smart TV vs Laptop
t_stat, p_value = ttest_ind(smart_tv_sub, laptop_sub)
if p_value < 0.05:
    print(f"Гипотезу о равенстве средних баллов подписки для Smart TV vs Laptop ОТВЕРГАЕМ: T-stat = {t_stat}, P-value = {p_value}")
else:
    print(f"Гипотезу о равенстве средних баллов подписки для Smart TV vs Laptop НЕ ОТВЕРГАЕМ: T-stat = {t_stat}, P-value = {p_value}")


Гипотезу о равенстве средних баллов подписки для Smartphone vs Tablet НЕ ОТВЕРГАЕМ: T-stat = 0.0890811531188364, P-value = 0.9290175958953631
Гипотезу о равенстве средних баллов подписки для Smartphone vs Smart TV НЕ ОТВЕРГАЕМ: T-stat = -0.7671697411109015, P-value = 0.4429824373596223
Гипотезу о равенстве средних баллов подписки для Smartphone vs Laptop НЕ ОТВЕРГАЕМ: T-stat = -0.31126140467535574, P-value = 0.75560256222922
Гипотезу о равенстве средних баллов подписки для Tablet vs Smart TV НЕ ОТВЕРГАЕМ: T-stat = -0.8568853140294846, P-value = 0.3915103504094646
Гипотезу о равенстве средних баллов подписки для Tablet vs Laptop НЕ ОТВЕРГАЕМ: T-stat = -0.40070292706509003, P-value = 0.6886397214098645
Гипотезу о равенстве средних баллов подписки для Smart TV vs Laptop НЕ ОТВЕРГАЕМ: T-stat = 0.4567960726859344, P-value = 0.6478186349328845


Итак, все гипотезы о равенстве средних баллов подписки для пар устройств не отвергнуты. Это означает, что для всех сравниваемых устройств нет статистически значимой разницы в типах подписок, которые выбирают пользователи. Странно, что тип устройства не оказывает значительного влияния на выбор между различными планами подписки. Возможно, тут затесалась ошибка II рода.

### Сравнение №3 - для метрики Age (одинаков ли средний возраст пользователей для разных типов устройств?)

Нулевая гипотеза (H0): Средний возраст пользователей для всех типов устройств одинаков

Альтернативная гипотеза (H1): Средний возраст пользователей для разных типов устройств отличается

In [15]:
smartphone_age = df_analysis[df_analysis['Device'] == 'Smartphone']['Age']
tablet_age = df_analysis[df_analysis['Device'] == 'Tablet']['Age']
smart_tv_age = df_analysis[df_analysis['Device'] == 'Smart TV']['Age']
laptop_age = df_analysis[df_analysis['Device'] == 'Laptop']['Age']

# Попарные сравнения:
# Smartphone vs Tablet
t_stat, p_value = ttest_ind(smartphone_age, tablet_age)
if p_value < 0.05:
    print(f"Гипотезу о равенстве средних возрастов для Smartphone vs Tablet ОТВЕРГАЕМ: T-stat = {t_stat}, P-value = {p_value}")
else:
    print(f"Гипотезу о равенстве средних возрастов для Smartphone vs Tablet НЕ ОТВЕРГАЕМ: T-stat = {t_stat}, P-value = {p_value}")

# Smartphone vs Smart TV
t_stat, p_value = ttest_ind(smartphone_age, smart_tv_age)
if p_value < 0.05:
    print(f"Гипотезу о равенстве средних возрастов для Smartphone vs Smart TV ОТВЕРГАЕМ: T-stat = {t_stat}, P-value = {p_value}")
else:
    print(f"Гипотезу о равенстве средних возрастов для Smartphone vs Smart TV НЕ ОТВЕРГАЕМ: T-stat = {t_stat}, P-value = {p_value}")

# Smartphone vs Laptop
t_stat, p_value = ttest_ind(smartphone_age, laptop_age)
if p_value < 0.05:
    print(f"Гипотезу о равенстве средних возрастов для Smartphone vs Laptop ОТВЕРГАЕМ: T-stat = {t_stat}, P-value = {p_value}")
else:
    print(f"Гипотезу о равенстве средних возрастов для Smartphone vs Laptop НЕ ОТВЕРГАЕМ: T-stat = {t_stat}, P-value = {p_value}")


# Tablet vs Smart TV
t_stat, p_value = ttest_ind(tablet_age, smart_tv_age)
if p_value < 0.05:
    print(f"Гипотезу о равенстве средних возрастов для Tablet vs Smart TV ОТВЕРГАЕМ: T-stat = {t_stat}, P-value = {p_value}")
else:
    print(f"Гипотезу о равенстве средних возрастов для Tablet vs Smart TV НЕ ОТВЕРГАЕМ: T-stat = {t_stat}, P-value = {p_value}")

# Tablet vs Laptop
t_stat, p_value = ttest_ind(tablet_age, laptop_age)
if p_value < 0.05:
    print(f"Гипотезу о равенстве средних возрастов для Tablet vs Laptop ОТВЕРГАЕМ: T-stat = {t_stat}, P-value = {p_value}")
else:
    print(f"Гипотезу о равенстве средних возрастов для Tablet vs Laptop НЕ ОТВЕРГАЕМ: T-stat = {t_stat}, P-value = {p_value}")

# Smart TV vs Laptop
t_stat, p_value = ttest_ind(smart_tv_age, laptop_age)
if p_value < 0.05:
    print(f"Гипотезу о равенстве средних возрастов для Smart TV vs Laptop ОТВЕРГАЕМ: T-stat = {t_stat}, P-value = {p_value}")
else:
    print(f"Гипотезу о равенстве средних возрастов для Smart TV vs Laptop НЕ ОТВЕРГАЕМ: T-stat = {t_stat}, P-value = {p_value}")

Гипотезу о равенстве средних возрастов для Smartphone vs Tablet ОТВЕРГАЕМ: T-stat = -2.0993467193463204, P-value = 0.03578887618261964
Гипотезу о равенстве средних возрастов для Smartphone vs Smart TV НЕ ОТВЕРГАЕМ: T-stat = -0.36964929922005685, P-value = 0.711644606915435
Гипотезу о равенстве средних возрастов для Smartphone vs Laptop НЕ ОТВЕРГАЕМ: T-stat = 0.03292595999354181, P-value = 0.9737336974456101
Гипотезу о равенстве средних возрастов для Tablet vs Smart TV НЕ ОТВЕРГАЕМ: T-stat = 1.7231286855614838, P-value = 0.08486838866725113
Гипотезу о равенстве средних возрастов для Tablet vs Laptop ОТВЕРГАЕМ: T-stat = 2.12984240276614, P-value = 0.03318708916281248
Гипотезу о равенстве средних возрастов для Smart TV vs Laptop НЕ ОТВЕРГАЕМ: T-stat = 0.4020450841520482, P-value = 0.6876517181712771


Итак, результаты анализа показывают, что существует статистически значимое различие в среднем возрасте пользователей между устройствами смартфон и планшет, а также между планшет и Laptop. Для сравнения смартфон и планшет (T-stat = -2.099), средний возраст пользователей смартфон оказался ниже, чем у пользователей планшет, а для планшет и Laptop (T-stat = 2.13), средний возраст пользователей планшет оказался выше, чем у пользователей Laptop. Однако для остальных пар, таких как смартфон и Smart TV, смартфон и Laptop, планшет и Smart TV, а также Smart TV и Laptop, различий в среднем возрасте пользователей не было найдено. Это говорит о том, что пользователи этих устройств не сильно различаются по возрасту, но при этом такие устройства как смартфон и планшет имеют отличительные возрастные профили по сравнению с остальными типами устройств.

**Тест Манна-Уитни: проверю, есть ли статзначимые различия в распределениях выборок для каждого сегмента по попарному сравнению каждой из метрик?**

H_0: Распределения для двух групп одинаковы.

H_1: Распределения для двух групп различаются (одно распределение смещено относительно другого).


### Сравнение №1 - для метрики Monthly Revenue (для какого типа устройства какое распределение по доходу?)

In [18]:
# Попарные сравнения:
# Smartphone vs Tablet
t_stat, p_value = mannwhitneyu(smartphone_revenue, tablet_revenue)
if p_value < 0.05:
    print(f"Гипотезу о равенстве распределений по доходу для Smartphone vs Tablet ОТВЕРГАЕМ: T-stat = {t_stat}, P-value = {p_value}")
else:
    print(f"Гипотезу о равенстве распределений по доходу для Smartphone vs Tablet НЕ ОТВЕРГАЕМ: T-stat = {t_stat}, P-value = {p_value}")

# Smartphone vs Smart TV
t_stat, p_value = mannwhitneyu(smartphone_revenue, smart_tv_revenue)
if p_value < 0.05:
    print(f"Гипотезу о равенстве распределений по доходу для Smartphone vs Smart TV ОТВЕРГАЕМ: T-stat = {t_stat}, P-value = {p_value}")
else:
    print(f"Гипотезу о равенстве распределений по доходу для Smartphone vs Smart TV НЕ ОТВЕРГАЕМ: T-stat = {t_stat}, P-value = {p_value}")

# Smartphone vs Laptop
t_stat, p_value = mannwhitneyu(smartphone_revenue, laptop_revenue)
if p_value < 0.05:
    print(f"Гипотезу о равенстве распределений по доходу для Smartphone vs Laptop ОТВЕРГАЕМ: T-stat = {t_stat}, P-value = {p_value}")
else:
    print(f"Гипотезу о равенстве распределений по доходу для Smartphone vs Laptop НЕ ОТВЕРГАЕМ: T-stat = {t_stat}, P-value = {p_value}")


# Tablet vs Smart TV
t_stat, p_value = mannwhitneyu(tablet_revenue, smart_tv_revenue)
if p_value < 0.05:
    print(f"Гипотезу о равенстве распределений по доходу для Tablet vs Smart TV ОТВЕРГАЕМ: T-stat = {t_stat}, P-value = {p_value}")
else:
    print(f"Гипотезу о равенстве распределений по доходу для Tablet vs Smart TV НЕ ОТВЕРГАЕМ: T-stat = {t_stat}, P-value = {p_value}")

# Tablet vs Laptop
t_stat, p_value = mannwhitneyu(tablet_revenue, laptop_revenue)
if p_value < 0.05:
    print(f"Гипотезу о равенстве распределений по доходу для Tablet vs Laptop ОТВЕРГАЕМ: T-stat = {t_stat}, P-value = {p_value}")
else:
    print(f"Гипотезу о равенстве распределений по доходу для Tablet vs Laptop НЕ ОТВЕРГАЕМ: T-stat = {t_stat}, P-value = {p_value}")

# Smart TV vs Laptop
t_stat, p_value = mannwhitneyu(smart_tv_revenue, laptop_revenue)
if p_value < 0.05:
    print(f"Гипотезу о равенстве распределений по доходу для Smart TV vs Laptop ОТВЕРГАЕМ: T-stat = {t_stat}, P-value = {p_value}")
else:
    print(f"Гипотезу о равенстве распределений по доходу для Smart TV vs Laptop НЕ ОТВЕРГАЕМ: T-stat = {t_stat}, P-value = {p_value}")


Гипотезу о равенстве распределений по доходу для Smartphone vs Tablet НЕ ОТВЕРГАЕМ: T-stat = 1221281944.5, P-value = 0.8608764107582407
Гипотезу о равенстве распределений по доходу для Smartphone vs Smart TV НЕ ОТВЕРГАЕМ: T-stat = 1218474863.0, P-value = 0.5907196215917532
Гипотезу о равенстве распределений по доходу для Smartphone vs Laptop НЕ ОТВЕРГАЕМ: T-stat = 1215665674.5, P-value = 0.299218923193472
Гипотезу о равенстве распределений по доходу для Tablet vs Smart TV НЕ ОТВЕРГАЕМ: T-stat = 1220777991.5, P-value = 0.4756117173493536
Гипотезу о равенстве распределений по доходу для Tablet vs Laptop НЕ ОТВЕРГАЕМ: T-stat = 1217962120.0, P-value = 0.3874893104502588
Гипотезу о равенстве распределений по доходу для Smart TV vs Laptop НЕ ОТВЕРГАЕМ: T-stat = 1208898756.0, P-value = 0.11568267142186252


### Сравнение №2 - для метрики Subscription Type

In [19]:
# Попарные сравнения:
# Smartphone vs Tablet
t_stat, p_value = mannwhitneyu(smartphone_sub, tablet_sub)
if p_value < 0.05:
    print(f"Гипотезу о равенстве распределений для баллов подписки для Smartphone vs Tablet ОТВЕРГАЕМ: T-stat = {t_stat}, P-value = {p_value}")
else:
    print(f"Гипотезу о равенстве распределений для баллов подписки для Smartphone vs Tablet НЕ ОТВЕРГАЕМ: T-stat = {t_stat}, P-value = {p_value}")

# Smartphone vs Smart TV
t_stat, p_value = mannwhitneyu(smartphone_sub, smart_tv_sub)
if p_value < 0.05:
    print(f"Гипотезу о равенстве распределений для баллов подписки для Smartphone vs Smart TV ОТВЕРГАЕМ: T-stat = {t_stat}, P-value = {p_value}")
else:
    print(f"Гипотезу о равенстве распределений для баллов подписки для Smartphone vs Smart TV НЕ ОТВЕРГАЕМ: T-stat = {t_stat}, P-value = {p_value}")

# Smartphone vs Laptop
t_stat, p_value = mannwhitneyu(smartphone_sub, laptop_sub)
if p_value < 0.05:
    print(f"Гипотезу о равенстве распределений для баллов подписки для Smartphone vs Laptop ОТВЕРГАЕМ: T-stat = {t_stat}, P-value = {p_value}")
else:
    print(f"Гипотезу о равенстве распределений для баллов подписки для Smartphone vs Laptop НЕ ОТВЕРГАЕМ: T-stat = {t_stat}, P-value = {p_value}")


# Tablet vs Smart TV
t_stat, p_value = mannwhitneyu(tablet_sub, smart_tv_sub)
if p_value < 0.05:
    print(f"Гипотезу о равенстве распределений для баллов подписки для Tablet vs Smart TV ОТВЕРГАЕМ: T-stat = {t_stat}, P-value = {p_value}")
else:
    print(f"Гипотезу о равенстве распределений для баллов подписки для Tablet vs Smart TV НЕ ОТВЕРГАЕМ: T-stat = {t_stat}, P-value = {p_value}")

# Tablet vs Laptop
t_stat, p_value = mannwhitneyu(tablet_sub, laptop_sub)
if p_value < 0.05:
    print(f"Гипотезу о равенстве распределений для длядних баллов подпискидля Tablet vs Laptop ОТВЕРГАЕМ: T-stat = {t_stat}, P-value = {p_value}")
else:
    print(f"Гипотезу о равенстве распределений для баллов подписки для Tablet vs Laptop НЕ ОТВЕРГАЕМ: T-stat = {t_stat}, P-value = {p_value}")

# Smart TV vs Laptop
t_stat, p_value = mannwhitneyu(smart_tv_sub, laptop_sub)
if p_value < 0.05:
    print(f"Гипотезу о равенстве распределений для баллов подписки для Smart TV vs Laptop ОТВЕРГАЕМ: T-stat = {t_stat}, P-value = {p_value}")
else:
    print(f"Гипотезу о равенстве распределений для баллов подписки для Smart TV vs Laptop НЕ ОТВЕРГАЕМ: T-stat = {t_stat}, P-value = {p_value}")


Гипотезу о равенстве распределений для баллов подписки для Smartphone vs Tablet НЕ ОТВЕРГАЕМ: T-stat = 1222432939.5, P-value = 0.9292807108291957
Гипотезу о равенстве распределений для баллов подписки для Smartphone vs Smart TV НЕ ОТВЕРГАЕМ: T-stat = 1212870156.0, P-value = 0.44301672280061855
Гипотезу о равенстве распределений для баллов подписки для Smartphone vs Laptop НЕ ОТВЕРГАЕМ: T-stat = 1218937986.0, P-value = 0.7554906366273939
Гипотезу о равенстве распределений для баллов подписки для Tablet vs Smart TV НЕ ОТВЕРГАЕМ: T-stat = 1214014010.0, P-value = 0.391462539040478
Гипотезу о равенстве распределений для баллов подписки для Tablet vs Laptop НЕ ОТВЕРГАЕМ: T-stat = 1220089923.0, P-value = 0.6885770815351968
Гипотезу о равенстве распределений для баллов подписки для Smart TV vs Laptop НЕ ОТВЕРГАЕМ: T-stat = 1217760325.0, P-value = 0.647497971733717


### Сравнение №3 - для метрики Age (одинаков ли средний возраст пользователей для разных типов устройств?)

In [20]:
# Попарные сравнения:
# Smartphone vs Tablet
t_stat, p_value = mannwhitneyu(smartphone_age, tablet_age)
if p_value < 0.05:
    print(f"Гипотезу о равенстве распределений возрастов для Smartphone vs Tablet ОТВЕРГАЕМ: T-stat = {t_stat}, P-value = {p_value}")
else:
    print(f"Гипотезу о равенстве распределений возрастов для Smartphone vs Tablet НЕ ОТВЕРГАЕМ: T-stat = {t_stat}, P-value = {p_value}")

# Smartphone vs Smart TV
t_stat, p_value = mannwhitneyu(smartphone_age, smart_tv_age)
if p_value < 0.05:
    print(f"Гипотезу о равенстве распределений возрастов для Smartphone vs Smart TV ОТВЕРГАЕМ: T-stat = {t_stat}, P-value = {p_value}")
else:
    print(f"Гипотезу о равенстве распределений возрастов для Smartphone vs Smart TV НЕ ОТВЕРГАЕМ: T-stat = {t_stat}, P-value = {p_value}")

# Smartphone vs Laptop
t_stat, p_value = mannwhitneyu(smartphone_age, laptop_age)
if p_value < 0.05:
    print(f"Гипотезу о равенстве распределений возрастов для Smartphone vs Laptop ОТВЕРГАЕМ: T-stat = {t_stat}, P-value = {p_value}")
else:
    print(f"Гипотезу о равенстве распределений возрастов для Smartphone vs Laptop НЕ ОТВЕРГАЕМ: T-stat = {t_stat}, P-value = {p_value}")


# Tablet vs Smart TV
t_stat, p_value = mannwhitneyu(tablet_age, smart_tv_age)
if p_value < 0.05:
    print(f"Гипотезу о равенстве распределений возрастов для Tablet vs Smart TV ОТВЕРГАЕМ: T-stat = {t_stat}, P-value = {p_value}")
else:
    print(f"Гипотезу о равенстве распределений возрастов для Tablet vs Smart TV НЕ ОТВЕРГАЕМ: T-stat = {t_stat}, P-value = {p_value}")

# Tablet vs Laptop
t_stat, p_value = mannwhitneyu(tablet_age, laptop_age)
if p_value < 0.05:
    print(f"Гипотезу о равенстве распределений возрастов для Tablet vs Laptop ОТВЕРГАЕМ: T-stat = {t_stat}, P-value = {p_value}")
else:
    print(f"Гипотезу о равенстве распределений возрастов для Tablet vs Laptop НЕ ОТВЕРГАЕМ: T-stat = {t_stat}, P-value = {p_value}")

# Smart TV vs Laptop
t_stat, p_value = mannwhitneyu(smart_tv_age, laptop_age)
if p_value < 0.05:
    print(f"Гипотезу о равенстве распределений возрастов для Smart TV vs Laptop ОТВЕРГАЕМ: T-stat = {t_stat}, P-value = {p_value}")
else:
    print(f"Гипотезу о равенстве распределений возрастов для Smart TV vs Laptop НЕ ОТВЕРГАЕМ: T-stat = {t_stat}, P-value = {p_value}")

Гипотезу о равенстве распределений возрастов для Smartphone vs Tablet ОТВЕРГАЕМ: T-stat = 1212653453.0, P-value = 0.03598606339550802
Гипотезу о равенстве распределений возрастов для Smartphone vs Smart TV НЕ ОТВЕРГАЕМ: T-stat = 1214452502.5, P-value = 0.7116721405489648
Гипотезу о равенстве распределений возрастов для Smartphone vs Laptop НЕ ОТВЕРГАЕМ: T-stat = 1220401689.5, P-value = 0.9737129679181546
Гипотезу о равенстве распределений возрастов для Tablet vs Smart TV НЕ ОТВЕРГАЕМ: T-stat = 1225296963.0, P-value = 0.08645618607556863
Гипотезу о равенстве распределений возрастов для Tablet vs Laptop ОТВЕРГАЕМ: T-stat = 1231318072.5, P-value = 0.033491216531074466
Гипотезу о равенстве распределений возрастов для Smart TV vs Laptop НЕ ОТВЕРГАЕМ: T-stat = 1217630090.0, P-value = 0.6874961243942113


## Результаты теста Манна-Уитни полностью коррелируют с результатами теста Стьюдента

# Тест Фишера для нескольких сегментов

Для теста Фишера для нескольких сегментов нужно сделать таблицы сопряженности (будет новая таблица для каждой метрики)

Поскольку Monthly Revenue и Age - числовые переменные, сделаю их категориальными:

In [49]:
# Доход
df_analysis['Revenue Group'] = pd.cut(df_analysis['Monthly Revenue'], bins=[9, 12, 14, 15], labels=['Low', 'Medium', 'High'])
contingency_table_revenue = pd.crosstab(df_analysis['Device'], df_analysis['Revenue Group'])

# Возраст
def age_group(age):
    if age <= 30:
        return '18-30'
    elif age <= 50:
        return '31-50'
    else:
        return '51+'

df_analysis['Age Group'] = df_analysis['Age'].apply(age_group)

contingency_table_age = pd.crosstab(df_analysis['Device'], df_analysis['Age Group'])

# Тип подписки (Score)

contingency_table_subscription_score = pd.crosstab(df_analysis['Device'], df_analysis['Subscription Score'])

In [48]:
df_analysis

,User ID,Subscription Type,Monthly Revenue,Join Date,Last Payment Date,Country,Age,Gender,Device,Plan Duration,Subscription Score,Age Group,Revenue Group
0,1,Basic,10,15-01-22,10-06-23,United States,28,Male,Smartphone,1 Month,1,18-30,Low
1,2,Premium,15,05-09-21,22-06-23,Canada,35,Female,Tablet,1 Month,3,31-50,High
2,3,Standard,12,28-02-23,27-06-23,United Kingdom,42,Male,Smart TV,1 Month,2,31-50,Low
3,4,Standard,12,10-07-22,26-06-23,Australia,51,Female,Laptop,1 Month,2,51+,Low
4,5,Basic,10,01-05-23,28-06-23,Germany,33,Male,Smartphone,1 Month,1,31-50,Low
...,...,...,...,...,...,...,...,...,...,...,...,...,...
197495,197496,Premium,14,10-06-23,03-07-23,Italy,26,Male,Smartphone,1 Month,3,18-30,Medium
197496,197497,Premium,13,02-05-22,05-06-23,Brazil,45,Female,Tablet,1 Month,3,31-50,Medium
197497,197498,Basic,10,14-12-22,04-06-23,Spain,48,Male,Laptop,1 Month,1,31-50,Low
197498,197499,Premium,14,03-08-23,26-06-23,Mexico,29,Female,Laptop,1 Month,3,18-30,Medium


In [51]:
chi2_stat, p_value_rev, dof, expected = chi2_contingency(contingency_table_revenue)
if p_value_rev < 0.05:
    print(f"Хотя бы в одном месте в таблице есть статистически значимая зависимость между типом устройства и уровнем дохода, P-value = {p_value_rev}")
else:
    print(f"Гипотеза о независимости не отвергается, что означает, что уровень дохода не зависит от типа устройства, P-value = {p_value_rev}")


chi2_stat, p_value_age, dof, expected = chi2_contingency(contingency_table_age)
if p_value_age < 0.05:
    print(f"Хотя бы в одном месте в таблице есть статистически значимая зависимость между типом устройства и возрастной группой, P-value = {p_value_age}")
else:
    print(f"Гипотеза о независимости не отвергается, что означает, что возрастная группа не зависит от типа устройства, P-value = {p_value_age}")

chi2_stat, p_value_SS, dof, expected = chi2_contingency(contingency_table_subscription_score)

if p_value_SS < 0.05:
    print(f"Хотя бы в одном месте в таблице есть статистически значимая зависимость между типом устройства и типом подписки, P-value = {p_value_SS}")
else:
    print(f"Гипотеза о независимости не отвергается, что означает, что тип подписки не зависит от типа устройства, P-value = {p_value_SS}")

Гипотеза о независимости не отвергается, что означает, что уровень дохода не зависит от типа устройства, P-value = 0.7185551243219708
Гипотеза о независимости не отвергается, что означает, что возрастная группа не зависит от типа устройства, P-value = 0.20753272532118777
Гипотеза о независимости не отвергается, что означает, что тип подписки не зависит от типа устройства, P-value = 0.94423955136088


В целом, по всем тестам Фишера гипотеза о независимости не отвергается, что означает, что в данных нет статистически значимых различий между устройствами по уровням дохода, возрастным группам или типам подписки.

In [41]:
df_analysis['Revenue Group'] = pd.cut(df_analysis['Monthly Revenue'], bins=[0, 12, float('inf')], labels=['Low', 'High'])

df_analysis['Age Group'] = pd.cut(df_analysis['Age'], bins=[0, 30, float('inf')], labels=['Young', 'Old'])

# Фишер 2 на 2 для Monthly Revenue (Low / High)

In [46]:
# делаю таблицы 2х2 для теста Фишера:

# Smartphone vs Tablet
contingency_table_revenue_2x2_smartphone_tablet = pd.crosstab(df_analysis[df_analysis['Device'].isin(['Smartphone', 'Tablet'])]['Device'], df_analysis['Revenue Group'])

odds_ratio_revenue_smartphone_tablet, p_value_revenue_smartphone_tablet = fisher_exact(contingency_table_revenue_2x2_smartphone_tablet)

if p_value_revenue_smartphone_tablet < 0.05:
    print(f"Гипотезу о равенстве распределений для баллов подписки для Tablet vs Smart TV ОТВЕРГАЕМ: T-stat = {t_stat}, P-value = {p_value}")
else:
    print(f"Гипотезу о равенстве распределений для баллов подписки для Tablet vs Smart TV НЕ ОТВЕРГАЕМ: T-stat = {t_stat}, P-value = {p_value}")

# Smartphone vs Smart TV

contingency_table_revenue_2x2_smartphone_smarttv = pd.crosstab(df_analysis[df_analysis['Device'].isin(['Smartphone', 'Smart TV'])]['Device'], df_analysis['Revenue Group'])

# Smartphone vs Laptop

contingency_table_revenue_2x2_smartphone_laptop = pd.crosstab(df_analysis[df_analysis['Device'].isin(['Smartphone', 'Laptop'])]['Device'], df_analysis['Revenue Group'])

# Tablet vs Smart TV

contingency_table_revenue_2x2_tablet_smarttv = pd.crosstab(df_analysis[df_analysis['Device'].isin(['Tablet', 'Smart TV'])]['Device'], df_analysis['Revenue Group'])

# Tablet vs Laptop

contingency_table_revenue_2x2_tablet_laptop = pd.crosstab(df_analysis[df_analysis['Device'].isin(['Tablet', 'Laptop'])]['Device'], df_analysis['Revenue Group'])

# Smart TV vs Laptop

contingency_table_revenue_2x2_tablet_laptop = pd.crosstab(df_analysis[df_analysis['Device'].isin(['Tablet', 'Laptop'])]['Device'], df_analysis['Revenue Group'])

In [45]:

contingency_table_revenue = pd.crosstab(df_analysis['Device'], df_analysis['Revenue Group'])
contingency_table_age = pd.crosstab(df_analysis['Device'], df_analysis['Age Group'])
# таблица сопряженности для переменных Device и Subscription Score
contingency_table_sub = pd.crosstab(df_analysis['Device'], df_analysis['Subscription Score'])

Revenue Group,Low,High
Device,,
Smartphone,24729,24678
Tablet,24763,24706


In [40]:
contingency_table_revenue

Revenue Group,Low,Medium,High
Device,,,
Laptop,24567,16461,8368
Smart TV,24702,16345,8181
Smartphone,24729,16450,8228
Tablet,24763,16505,8201


In [39]:
# теперь применяю тест Фишера для нескольких сегментов

# для Income Group
odds_ratio_revenue, p_value_revenue = fisher_exact(contingency_table_revenue)
print("p-value для теста Фишера по доходу:", p_value_revenue)

# # для Subscription Score
# odds_ratio_subscription, p_value_subscription = fisher_exact(contingency_table_sub)
# print("p-value для теста Фишера по типу подписки:", p_value_subscription)

# # для Age Group
# odds_ratio_age, p_value_age = fisher_exact(contingency_table_age)
# print("p-value для теста Фишера по возрастной группе:", p_value_age)

ValueError: The input `table` must be of shape (2, 2).

Простите, я немного запуталась и только в конце поняла, что нужно было во втором пункте разбить на два сегмента, а потом уже только разбивать на 3... я переделаю, но пока до дедлайна отправляю этот вариант

Использовать другие статистические методы для многоклассовых данных: Для таблиц большего размера (например, 4x3), где требуется сравнение нескольких категорий, можно применить критерий хи-квадрат (Chi-squared test), который проверяет зависимость между несколькими категориями: